In [ ]:
import numpy as np
import sep

In [ ]:
from astropy.io import fits

import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

In [ ]:
fname = "../hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits"
hdu_list = fits.open(fname)


In [ ]:

image_data = fits.getdata(fname)
image_data = image_data.byteswap(inplace=True).newbyteorder()
print(type(image_data))
print(image_data.shape)

In [ ]:
m, s = np.mean(image_data[~mask]), np.std(image_data[~mask])
plt.imshow(image_data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower',)
plt.colorbar();
plt.savefig('/Users/labuser/Desktop/image_data.png')

# background subtraction

In [ ]:
bkg = sep.Background(image_data, mask=mask, bw=64, bh=64, fw=3, fh=3)

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
bkg_image = np.array(bkg)

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('/Users/labuser/Desktop/bkg_image.png')

In [ ]:
bkg_rms = bkg.rms()

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('/Users/labuser/Desktop/bkg_rms.png')

In [ ]:
data_sub = data - bkg

# Object Detection

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

In [ ]:
len(objects)
#deteced ~3500 sources

In [ ]:
from matplotlib.patches import Ellipse

fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.savefig('/Users/labuser/Desktop/data_sub.png')

#plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]), 
                width=6*objects['a'][i], 
                height=6*objects['b'][i], 
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

In [ ]:
objects.dtype.names

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

# Histogram fluxes


In [ ]:
Histogram = plt.hist(image_data.flatten(), bins='auto')

In [ ]:
from matplotlib.colors import LogNorm

In [ ]:
plt.imshow(image_data, cmap='gray', norm=LogNorm())

cbar = plt.colorbar(ticks=[5.e3, 1.e4, 2.e4])
cbar.ax.set_yticklabels(['5,000', '10,000', '20,000'])